# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
# !pip -q install ./python

In [2]:
import gym
# !pip3 install box2d
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 

# !python -m pip install pyvirtualdisplay
# from pyvirtualdisplay import Display
# display = Display(visible=0, size=(1400, 900))
# display.start()

# is_ipython = 'inline' in plt.get_backend()
# if is_ipython:
#     from IPython import display

# plt.ion()

In [3]:
# !python -m ipykernel install --user --name drlnd --display-name "drlnd"


In [4]:
from unityagents import UnityEnvironment
import numpy as np


The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [4]:
# please do not modify the line below
file_path = r"E:\Bats_wall_following\RLecho\Navigation-DeepRL\Unity_sim_banana\Banana_Windows_x86_64"
env = UnityEnvironment(file_name= file_path+"\Banana.exe")
# env = UnityEnvironment(file_name="C:\Reinforcement\Banana\deep-reinforcement-learning\p1_navigation\Banana_Windows_x86_64\Banana_Windows_x86_64\Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [5]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [6]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [7]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    #print(len(next_state))
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


In [8]:
from dqn_agent_mod import Agent
agent = Agent(state_size=3*37, action_size=4, seed=0)

the device that will be used is cuda:0


In [9]:
# !pip install torchsummary
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0] 

In [10]:
from torchsummary import summary
summary(agent.qnetwork_local , (3*37,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]           7,168
            Linear-2                   [-1, 64]           4,160
            Linear-3                    [-1, 4]             260
Total params: 11,588
Trainable params: 11,588
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0.05
----------------------------------------------------------------


In [11]:
print(state -next_state)

[-1.          0.          1.          0.         -0.27860732  0.
  0.          0.          0.         -0.14827031  0.          0.
  0.          0.         -0.18736115  0.          0.          0.
  0.         -0.37723076  0.          0.          0.          0.
 -0.21509115  0.          0.          0.          0.         -0.05984911
  0.          0.          0.          0.         -0.31725508  0.
  0.        ]


In [12]:
state_type = type(state)
print(len(state))
test_agent = Agent(state_size=3*37, action_size=4, seed=0)
for i in range(65):
    test_agent.step(state+i, action, reward, next_state+i, False)


37


In [13]:
print(len(test_agent.memory.memory), done , test_agent.memory.memory[-1].state)
sampled_mem = test_agent.memory.sample_idx_withBatch(2)
print(sampled_mem[0].reshape(2,3*37).size())

test_agent.act(state, debug= True)

65 True [64.         64.         65.         64.         64.16101955 65.
 64.         64.         64.         64.04571758 65.         64.
 64.         64.         64.2937662  64.         64.         65.
 64.         64.14386636 64.         64.         65.         64.
 64.16776823 65.         64.         64.         64.         64.04420976
 65.         64.         64.         64.         64.05423063 64.
 64.        ]
torch.Size([2, 111])
torch.Size([1, 111]) tensor([[6.3000e+01, 6.3000e+01, 6.4000e+01, 6.3000e+01, 6.3161e+01, 6.4000e+01,
         6.3000e+01, 6.3000e+01, 6.3000e+01, 6.3046e+01, 6.4000e+01, 6.3000e+01,
         6.3000e+01, 6.3000e+01, 6.3294e+01, 6.3000e+01, 6.3000e+01, 6.4000e+01,
         6.3000e+01, 6.3144e+01, 6.3000e+01, 6.3000e+01, 6.4000e+01, 6.3000e+01,
         6.3168e+01, 6.4000e+01, 6.3000e+01, 6.3000e+01, 6.3000e+01, 6.3044e+01,
         6.4000e+01, 6.3000e+01, 6.3000e+01, 6.3000e+01, 6.3054e+01, 6.3000e+01,
         6.3000e+01, 6.4000e+01, 6.4000e+01, 6.5000e

0

In [14]:


def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        #print("hello")
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action.astype(int))[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            #print(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 1 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=16.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            if np.mean(scores_window)>=16.0:
                break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 1	Average Score: 2.00
Episode 2	Average Score: 2.50
Episode 3	Average Score: 1.67
Episode 4	Average Score: 1.25
Episode 5	Average Score: 1.20
Episode 6	Average Score: 1.67
Episode 7	Average Score: 1.43
Episode 8	Average Score: 1.25
Episode 9	Average Score: 0.89
Episode 10	Average Score: 0.90
Episode 11	Average Score: 0.82
Episode 12	Average Score: 0.92
Episode 13	Average Score: 0.77
Episode 14	Average Score: 0.71
Episode 15	Average Score: 0.67
Episode 16	Average Score: 0.69
Episode 17	Average Score: 0.65
Episode 18	Average Score: 0.56
Episode 19	Average Score: 0.42
Episode 20	Average Score: 0.40
Episode 21	Average Score: 0.38
Episode 22	Average Score: 0.45
Episode 23	Average Score: 0.43
Episode 24	Average Score: 0.42
Episode 25	Average Score: 0.40
Episode 26	Average Score: 0.42
Episode 27	Average Score: 0.37
Episode 28	Average Score: 0.36
Episode 29	Average Score: 0.34
Episode 30	Average Score: 0.33
Episode 31	Average Score: 0.29
Episode 32	Average Score: 0.28
Episode 33	Averag

Episode 261	Average Score: 5.30
Episode 262	Average Score: 5.27
Episode 263	Average Score: 5.37
Episode 264	Average Score: 5.36
Episode 265	Average Score: 5.34
Episode 266	Average Score: 5.39
Episode 267	Average Score: 5.42
Episode 268	Average Score: 5.43
Episode 269	Average Score: 5.45
Episode 270	Average Score: 5.41
Episode 271	Average Score: 5.38
Episode 272	Average Score: 5.41
Episode 273	Average Score: 5.46
Episode 274	Average Score: 5.45
Episode 275	Average Score: 5.54
Episode 276	Average Score: 5.61
Episode 277	Average Score: 5.71
Episode 278	Average Score: 5.79
Episode 279	Average Score: 5.81
Episode 280	Average Score: 5.87
Episode 281	Average Score: 5.89
Episode 282	Average Score: 5.92
Episode 283	Average Score: 6.02
Episode 284	Average Score: 5.98
Episode 285	Average Score: 6.02
Episode 286	Average Score: 6.01
Episode 287	Average Score: 6.01
Episode 288	Average Score: 6.02
Episode 289	Average Score: 6.09
Episode 290	Average Score: 6.04
Episode 291	Average Score: 6.09
Episode 

Episode 515	Average Score: 12.69
Episode 516	Average Score: 12.73
Episode 517	Average Score: 12.78
Episode 518	Average Score: 12.78
Episode 519	Average Score: 12.83
Episode 520	Average Score: 12.89
Episode 521	Average Score: 12.75
Episode 522	Average Score: 12.72
Episode 523	Average Score: 12.76
Episode 524	Average Score: 12.78
Episode 525	Average Score: 12.74
Episode 526	Average Score: 12.72
Episode 527	Average Score: 12.70
Episode 528	Average Score: 12.76
Episode 529	Average Score: 12.87
Episode 530	Average Score: 12.85
Episode 531	Average Score: 12.79
Episode 532	Average Score: 12.85
Episode 533	Average Score: 12.87
Episode 534	Average Score: 12.99
Episode 535	Average Score: 13.05
Episode 536	Average Score: 13.18
Episode 537	Average Score: 13.16
Episode 538	Average Score: 13.13
Episode 539	Average Score: 13.13
Episode 540	Average Score: 13.16
Episode 541	Average Score: 13.18
Episode 542	Average Score: 13.26
Episode 543	Average Score: 13.29
Episode 544	Average Score: 13.33
Episode 54

Episode 764	Average Score: 15.04
Episode 765	Average Score: 15.10
Episode 766	Average Score: 15.06
Episode 767	Average Score: 15.03
Episode 768	Average Score: 15.08
Episode 769	Average Score: 15.09
Episode 770	Average Score: 15.20
Episode 771	Average Score: 15.14
Episode 772	Average Score: 15.10
Episode 773	Average Score: 15.12
Episode 774	Average Score: 15.09
Episode 775	Average Score: 15.14
Episode 776	Average Score: 15.10
Episode 777	Average Score: 15.19
Episode 778	Average Score: 15.14
Episode 779	Average Score: 15.18
Episode 780	Average Score: 15.14
Episode 781	Average Score: 15.10
Episode 782	Average Score: 15.07
Episode 783	Average Score: 15.07
Episode 784	Average Score: 15.05
Episode 785	Average Score: 15.03
Episode 786	Average Score: 15.01
Episode 787	Average Score: 14.95
Episode 788	Average Score: 14.96
Episode 789	Average Score: 14.97
Episode 790	Average Score: 14.89
Episode 791	Average Score: 14.89
Episode 792	Average Score: 14.86
Episode 793	Average Score: 14.72
Episode 79

KeyboardInterrupt: 

In [18]:
# load the weights from file
# running the trained model specfic to unity 
# **agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]
for i in range(3):
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]
    score =0
#     img = plt.imshow(enstate = env.reset()v.render(mode='rgb_array'))
    for j in range(300):
        
        action = agent.act(state,eps=0.2)
        env_info = env.step(action.astype(int))[brain_name]
        state = env_info.vector_observations[0]
        done = env_info.local_done[0]
        reward = env_info.rewards[0]
        score += reward
#         img.set_data(env.render(mode='rgb_array')) 
#         plt.axis('off')
#         display.display(plt.gcf())
#         display.clear_output(wait=True)
#         state, reward, done, _ = env.step(action)
        if done:
            print(score)
            score =0
            break 
            
# env.close()

5.0
5.0
10.0


When finished, you can close the environment.

In [11]:
# env.close()

In [ ]:
# this is second trying use the same agent to solve cart pole problem

In [19]:
env1 = gym.make('CartPole-v0')

In [20]:
print("the enviroment observtion space", env1.observation_space ,"action space", env1.action_space)

the enviroment observtion space Box(4,) action space Discrete(2)


In [21]:
action = 1
env1.reset()
observation, reward, done, info = env1.step(action)
print("the observation size", len(observation), "the reduced observation", observation)
state_Type = type(observation)
print("the state_type of the observation", state_Type , observation)

the observation size 4 the reduced observation [-0.04705177  0.14745282  0.03203246 -0.27001715]
the state_type of the observation <class 'numpy.ndarray'> [-0.04705177  0.14745282  0.03203246 -0.27001715]


In [22]:
action_size = 2
score = 0 # initialize the score
i = 30
while i > 0:
    action = np.random.randint(action_size)               # select an action
    observation, reward, done, info = env1.step(action)      # send the action to the environment
    next_state = observation                                # see if episode has finished
    
    
    score += reward                                         # update the score
    state = next_state                                      # roll over the state to next time step
    i-=1
    if done:                                                # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 30.0


In [23]:
len(observation)

4

In [29]:


pen_agent = Agent(state_size=3*len(observation), action_size = 2 , seed = 5 , augment_frames= 1)

summary(pen_agent.qnetwork_local , (12,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             832
            Linear-2                   [-1, 64]           4,160
            Linear-3                    [-1, 2]             130
Total params: 5,122
Trainable params: 5,122
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


In [49]:
def dqn(n_episodes=3000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env1.reset()
        score = 0
        #print("hello")
        for t in range(max_t):
            action = pen_agent.act(state, eps)
            next_state, reward, done, info = env1.step(action)      # send the action to the environment
            pen_agent.step(state, action, reward, next_state, done)
            #print(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 1 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=180.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'pen_checkpoint.pth')
            if np.mean(scores_window)>=180.0:
                break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 1	Average Score: 8.00
Episode 2	Average Score: 14.00
Episode 3	Average Score: 14.67
Episode 4	Average Score: 14.25
Episode 5	Average Score: 16.40
Episode 6	Average Score: 16.50
Episode 7	Average Score: 15.71
Episode 8	Average Score: 15.00
Episode 9	Average Score: 15.78
Episode 10	Average Score: 15.30
Episode 11	Average Score: 15.36
Episode 12	Average Score: 15.50
Episode 13	Average Score: 19.00
Episode 14	Average Score: 18.36
Episode 15	Average Score: 19.33
Episode 16	Average Score: 19.00
Episode 17	Average Score: 18.82
Episode 18	Average Score: 19.00
Episode 19	Average Score: 21.26
Episode 20	Average Score: 22.30
Episode 21	Average Score: 21.76
Episode 22	Average Score: 22.73
Episode 23	Average Score: 23.35
Episode 24	Average Score: 23.54
Episode 25	Average Score: 23.24
Episode 26	Average Score: 23.35
Episode 27	Average Score: 24.15
Episode 28	Average Score: 24.25
Episode 29	Average Score: 24.00
Episode 30	Average Score: 25.13
Episode 31	Average Score: 25.23
Episode 32	Average

KeyboardInterrupt: 

In [54]:
action_size = 2
score = 0 # initialize the score
i = 10000
state = env1.reset()
while i > 0:
    action = pen_agent.act(state , eps =0.)                 # select an action
    next_state, reward, done, info = env1.step(action)      # send the action to the environment
    state = next_state                                # see if episode has finished
    
    
    score += reward                                         # update the score
    state = next_state
    env1.render(mode='rgb_array')
    # roll over the state to next time step
    i-=1
    if done:                                                # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 200.0
